<img src='./img/logoline_12000.png' align='right' width='100%'></img>

<br>

# Bias Correction of seasonal forecast data

### About

This notebook provides you a practical introduction ..


The notebook has the following outline:

* [1 - Data preparation](#data_preparation)
* [2 - Calculation of the annual global mean surface air temperature anomaly](#global_mean)
* [3 - Calculation of the monthly surface air temperature anomaly in Europe](#monthly_europe)
* [4 - Calculation of seasonal variations of near-surface air temperature over the Arctic](#seasonal_arctic)

### Data

This notebook introduces you to the ERA5 reanalysis monthly averaged data on single levels. The data has the following specifications:

> **Data**: `ERA5 monthly averaged data on single levels` <br>
> **Temporal coverage**: `Jan 1979 to Dec 2020` <br>
> **Spatial coverage**: `Global` <br>
> **Format**: `NetCDF`<br>
> **Retrieve request**: [CDS API request](./100_climate_data_store_intro.ipynb#era5_monthly)

Get [here](./101_c3s_data_intro.ipynb) more information about climate reanalysis data in general including an overview of they different dataset types.

### Further resources

<br>

<hr>

### Load libraries

In [2]:
# CDS API
import cdsapi

# Libraries for working with multi-dimensional arrays
import numpy as np
import xarray as xr
import pandas as pd

# Libraries for plotting and visualising data
import matplotlib.path as mpath
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

<hr>

## <a id='data_preparation'></a>1. Data preparation

The example below will show you how you can use ERA5 reanalysis data to analyse climatologies and anomalies of the global near-surface air temperature.

## Retrieve SEAS5 monthly hindcast data - Sea surface temperature

In [3]:
c = cdsapi.Client()

c.retrieve(
    'seasonal-monthly-single-levels',
    {
        'format': 'grib',
        'originating_centre': 'ecmwf',
        'system': '5',
        'variable': 'sea_surface_temperature',
        'product_type': 'monthly_mean',
        'year': [
            '1993', '1994', '1995',
            '1996', '1997', '1998',
            '1999', '2000', '2001',
            '2002', '2003', '2004',
            '2005', '2006', '2007',
            '2008', '2009', '2010',
            '2011', '2012', '2013',
            '2014', '2015', '2016',
        ],
        'month': '09',
        'leadtime_month': [
            '1', '2', '3',
            '4', '5', '6',
        ],
    },
    './ecmwf_seas5_1993-2016_09_hindcast_monthly_sst.grib')

2021-10-12 09:39:39,602 INFO Welcome to the CDS
2021-10-12 09:39:39,603 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2021-10-12 09:39:39,821 INFO Request is queued
2021-10-12 09:40:00,979 INFO Request is running
2021-10-12 09:41:33,804 INFO Request is completed
2021-10-12 09:41:33,806 INFO Downloading https://download-0011.copernicus-climate.eu/cache-compute-0011/cache/data0/adaptor.mars.external-1634024398.5116434-17563-9-a5fdf7bf-2668-457d-a69a-3f526ec99a20.grib to ./ecmwf_seas5_1993-2016_09_hindcast_monthly_sst.grib (324.2M)
2021-10-12 09:42:46,726 INFO Download rate 4.4M/s  


Result(content_length=339984000,content_type=application/x-grib,location=https://download-0011.copernicus-climate.eu/cache-compute-0011/cache/data0/adaptor.mars.external-1634024398.5116434-17563-9-a5fdf7bf-2668-457d-a69a-3f526ec99a20.grib)

## Retrieve SEAS5 monthly forecast data for september 2021 - Sea surface temperature

In [4]:
c = cdsapi.Client()

c.retrieve(
    'seasonal-monthly-single-levels',
    {
        'format': 'grib',
        'originating_centre': 'ecmwf',
        'system': '5',
        'variable': 'sea_surface_temperature',
        'product_type': 'monthly_mean',
        'year': '2021',
        'month': '09',
        'leadtime_month': [
            '1', '2', '3',
            '4', '5', '6',
        ],
    },
    './data/ecmwf_seas5_2021_09_forecast_monthly_sst.grib')

2021-10-12 09:43:25,024 INFO Welcome to the CDS
2021-10-12 09:43:25,024 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2021-10-12 09:43:25,515 INFO Request is queued
2021-10-12 09:44:15,214 INFO Request is completed
2021-10-12 09:44:15,216 INFO Downloading https://download-0011.copernicus-climate.eu/cache-compute-0011/cache/data2/adaptor.mars.external-1634024646.1267226-3795-20-24b705cd-2242-412c-ab77-8cddd969ce3e.grib to ./data/ecmwf_seas5_2021_09_forecast_monthly_sst.grib (27.6M)
2021-10-12 09:44:21,497 INFO Download rate 4.4M/s   


Result(content_length=28898640,content_type=application/x-grib,location=https://download-0011.copernicus-climate.eu/cache-compute-0011/cache/data2/adaptor.mars.external-1634024646.1267226-3795-20-24b705cd-2242-412c-ab77-8cddd969ce3e.grib)

## Retrieve ERA5 monthly (observations) data for September from 1993 to 2016

### Load the dataset

<hr>

<p><img src='./img/copernicus_logo.png' align='right' alt='Logo EU Copernicus' width='20%'></img></p>
<br><br><br><br><br>
<span style='float:right'><p style=\"text-align:right;\">This project is licensed under <a href="./LICENSE">APACHE License 2.0</a>. | <a href=\"https://github.com/ecmwf-projects/copernicus-training">View on GitHub</a></span>